#  Install package

In [1]:
# ! pip install pyvis
# ! pip install networkx
# ! pip install openai
# ! pip install contextily
# ! pip install matplotlib_scalebar
# ! pip install langchain
# ! pip install langchain-openai
# ! pip install geojson
# ! pip install OSMPythonTools

# Import package

In [2]:
%load_ext autoreload
%autoreload 2
    
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
# from pyvis.network import Network
from openai import OpenAI
from IPython.display import display, HTML, Code
from IPython.display import clear_output

import base64

import pickle
import sys
import osmnx as ox



import LLM_Find_Constants as constants
import helper

from LLM_Find_kernel import Solution

from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate





In [3]:

OpenAI_key = helper.load_OpenAI_key()
# llm = ChatOpenAI(api_key=OpenAI_key, model="gpt-4o", temperature=0)

## Input task and data desciption

In [4]:
# isReview = True
isReview = False

In [5]:
# task

In [6]:
# # Case 1
# task_name ='OSM_PA_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_PA_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of Pennsylvania State, USA from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_SC_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_SC_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of South Carolina State, USA from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_PA_hospital'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_PA_hospital.gpkg.'
# task = rf'''1. Download all hospitals in Pennsylvania State, USA, from OpenStreetMap.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_SC_hospital'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_SC_hospital.gpkg.'
# task = rf'''1. Download all hospitals in South Carolina State, USA, from OpenStreetMap.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_SC_school'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_SC_school.gpkg.'
# task = rf'''1. Download all schools in South Carolina State, USA, from OpenStreetMap.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_CA_park'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_CA_parks.gpkg.'
# task = rf'''1. Download all parks in California, USA, from OpenStreetMap, including urban public, recreation, state, and national parks.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# # '''

# task_name ='OSM_USA_university'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_USA_universities.gpkg.'
# task = rf'''1. Download all universities and colleges in USA from OpenStreetMap.
# 2. Save the downloaded data as points in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_state_college_street'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_state_college_street.gpkg.'
# task = rf'''1. Download all streets of State College, Pennsylvania, USA, from OpenStreetMap.
# 2. Save the downloaded data as polylines in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_tract'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_tract.gpkg.'
# if os.path.exists(saved_fname):
#     os.remove(saved_fname)
# task = rf'''1. Download all census tract boundaries in South Carolina, USA.
# 2. Save the downloaded data as polygons in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_blockgroups'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_blockgroups.gpkg.'
# if os.path.exists(saved_fname):
#     os.remove(saved_fname)
# task = rf'''1. Download all Census block group boundaries in South Carolina, USA.
# 2. Save the downloaded data as polygons in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='OSM_Nigeria_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_Nigeria_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of Nigeria from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''


# task_name ='OSM_Afghanistan_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_Afghanistan_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of Afghanistan from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# Case 1
# task_name ='Census_Centre_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_Centre_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of Centre County of Pennsylvania State, USA from Census Bureau.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_countries_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_counties_boundary.gpkg.'
# task = rf'''1. Download the administrative boundary of all Counties of South Carolina from Census Bureau.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# # This is a difficult case, the above cases were okay during the test.
# task_name ='Wuhan_railway_network'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Wuhan_Railway_network.gpkg.'
# task = rf'''1. Download the railway network in Wuhan, Hubei, China.
# 2. Save the downloaded data as polylines in GeoPackage format, save it at: {saved_fname} 
# '''
#  Using "Hubei Province" may not return polygons.


# task_name ='OSM_Nigeria_railway'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\OSM_Nigeria_railway.gpkg.'
# task = rf'''1. Download the railway network of Nigeria from OpenStreetMap.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_counties_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_counties_population.csv'
# task = rf'''1. Download latest population for each county in South Carolina.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_Richland_race_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_Richland_race_population.csv'
# task = rf'''1. Download latest population by race for Richland county in South Carolina.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_PA_counties_race_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_PA_counties_race_population.csv'
# task = rf'''1. Download latest population by race for all counties in Pennsylvania.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''


task_name ='Census_US_states_education'
saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_states_education.csv'
task = rf'''1. Download latest population by education attainment for all states in USA, also the total population of each state.
2. Save the downloaded data as CSV files, save it at: {saved_fname} 
'''


save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)
# 
# model=r"gpt-4"
# model = r'gpt-4-turbo-preview'
# model_name = "gpt-3.5-turbo"


model_name = r'gpt-4o'
# model_name = r'gpt-4'


model = ChatOpenAI(api_key=OpenAI_key, model=model_name, temperature=1, stream=True)

# model

e:\ProgramData\Anaconda3\envs\ox\Lib\site-packages\langchain_core\utils\utils.py:161: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  warnings.warn(


In [7]:
%load_ext autoreload
%autoreload 2
import helper
    
source_select_prompt_str = helper.create_select_prompt(task=task)

# solution = Solution(
#                     task=TASK,
#                     task_name=task_name,
#                     save_dir=save_dir,
#                     data_locations=DATA_LOCATIONS,
#                     model=model,
#                     )
# print("Prompt to select data source:\n")
# print(source_select_prompt_langChain.dict()['messages'][1]['prompt']['template'])
print(source_select_prompt_str)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Your role: A professional Python programmer in geographic information science (GIScience). You have worked on GIScience for more than 20 years and know every detail and pitfall when collecting data and coding. You know which websites you can get suitable spatial data and know the methods or tricks to download data, such as OpenStreetMap, Census Bureau, or various APIs. You are also experienced in processing the downloaded data, including saving them in suitable formats, map projections, and creating detailed and useful meta-data.
 
Your mission: select a suitable data source from the given list to download the requested geo-spatial data for this task: 1. Download latest population by education attainment for all states in USA, also the total population of each state.
2. Save the downloaded data as CSV files, save it at: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Fi

In [8]:
# print("")

from IPython.display import clear_output

chunks = []

async for chunk in model.astream(source_select_prompt_str):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

# clear_output(wait=True)
clear_output(wait=False)
LLM_reply_str = helper.convert_chunks_to_str(chunks=chunks)
print(LLM_reply_str)

{'Explaination': "For downloading the latest population by education attainment for all states in the USA, including the total population, the most suitable data source provided is the US Census Bureau demography.", "Selected data source": 'US Census Bureau demography'}


In [9]:
import ast
select_source = ast.literal_eval(LLM_reply_str)

selected_data_source= select_source['Selected data source']
data_source_ID = constants.data_source_dict[selected_data_source]['ID']

print("selected_data_source:", selected_data_source)
print("data_source_ID:", data_source_ID)

handbook_list = constants.handbooks[f"{data_source_ID}"]
handbook_str =  '\n'.join([f"{idx + 1}. {line}" for idx, line in enumerate(handbook_list)])
print()
print(f"Handbook:\n{handbook_str}")

selected_data_source: US Census Bureau demography
data_source_ID: US_Census_demography

Handbook:
1. If you need an API key, you can use this: b12026d61228a4b0d441ae7aa93f1ea222877503
2. Store the returns into CSV files, please make sure using descriptive headers without special characters.
3. Add the year of the data to the saved CSV files.
4. Prefer the office APIs, do not use other Python pacakges such as `census`.


In [10]:
# constants.handbooks[f"US_Census"]

%load_ext autoreload
%autoreload 2

download_prompt_str = helper.create_download_prompt(task, selected_data_source, handbook_str)

# solution = Solution(
#                     task=TASK,
#                     task_name=task_name,
#                     save_dir=save_dir,
#                     data_locations=DATA_LOCATIONS,
#                     model=model,
#                     )
# print("Prompt to select data source:\n")
# print(source_select_prompt_langChain.dict()['messages'][1]['prompt']['template'])
print(download_prompt_str)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Your role: A professional Python programmer in geographic information science (GIScience). You have worked on GIScience for more than 20 years and know every detail and pitfall when collecting data and coding. You know which websites you can get suitable spatial data and know the methods or tricks to download data, such as OpenStreetMap, Census Bureau, or various APIs. You are also experienced in processing the downloaded data, including saving them in suitable formats, map projections, and creating detailed and useful meta-data. When downloading geo-spatial data, the handbook for a perticular data source is provided, you can follow it, and write Python code carefully to download the data. 
 
Your mission: download geo-spatial data from the given data source for this task: 1. Download latest population by education attainment for all states in USA, also the total population of each state.
2. Save th

In [11]:
from IPython.display import clear_output

chunks = []

async for chunk in model.astream(download_prompt_str):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

# clear_output(wait=True)
clear_output(wait=False)
LLM_reply_str = helper.convert_chunks_to_str(chunks=chunks)
print(LLM_reply_str)

Certainly! Here is the required Python code to download the latest population by education attainment for all states in the USA from the US Census Bureau demography API, and then save it to a CSV file:

```python
import requests
import pandas as pd

def download_data():
    # API base URL
    base_url = "https://api.census.gov/data/2021/acs/acs1"
    
    # API key
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    
    # Specify the variables we are interested in
    # B15003_001E: Total Population (Education Attainment)
    # B15003_002E to B15003_017E: Population by specific education attainment (less than 9th grade to Doctorate degree)
    variables = [
        "NAME", "B15003_001E", "B15003_002E", "B15003_003E", "B15003_004E", 
        "B15003_005E", "B15003_006E", "B15003_007E", "B15003_008E", 
        "B15003_009E", "B15003_010E", "B15003_011E", "B15003_012E", 
        "B15003_013E", "B15003_014E", "B15003_015E", "B15003_016E", "B15003_017E"
    ]

    # Construct the 

In [12]:
code = helper.extract_code_from_str(LLM_reply_str, task)
display(Code(code, language='python'))

import requests
import pandas as pd

def download_data():
    # API base URL
    base_url = "https://api.census.gov/data/2021/acs/acs1"
    
    # API key
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    
    # Specify the variables we are interested in
    # B15003_001E: Total Population (Education Attainment)
    # B15003_002E to B15003_017E: Population by specific education attainment (less than 9th grade to Doctorate degree)
    variables = [
        "NAME", "B15003_001E", "B15003_002E", "B15003_003E", "B15003_004E", 
        "B15003_005E", "B15003_006E", "B15003_007E", "B15003_008E", 
        "B15003_009E", "B15003_010E", "B15003_011E", "B15003_012E", 
        "B15003_013E", "B15003_014E", "B15003_015E", "B15003_016E", "B15003_017E"
    ]

    # Construct the query URL
    url = f"{base_url}?get={','.join(variables)}&for=state:*&key={api_key}"

    # Request the data
    response = requests.get(url)
    if response.status_code == 200:
        # Convert to JSON
        

import requests
import pandas as pd

def download_data():
    # API base URL
    base_url = "https://api.census.gov/data/2021/acs/acs1"
    
    # API key
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    
    # Specify the variables we are interested in
    # B15003_001E: Total Population (Education Attainment)
    # B15003_002E to B15003_017E: Population by specific education attainment (less than 9th grade to Doctorate degree)
    variables = [
        "NAME", "B15003_001E", "B15003_002E", "B15003_003E", "B15003_004E", 
        "B15003_005E", "B15003_006E", "B15003_007E", "B15003_008E", 
        "B15003_009E", "B15003_010E", "B15003_011E", "B15003_012E", 
        "B15003_013E", "B15003_014E", "B15003_015E", "B15003_016E", "B15003_017E"
    ]

    # Construct the query URL
    url = f"{base_url}?get={','.join(variables)}&for=state:*&key={api_key}"

    # Request the data
    response = requests.get(url)
    if response.status_code == 200:
        # Convert to JSON
        data = response.json()
        
        # Create a DataFrame from the received data
        df = pd.DataFrame(data[1:], columns=data[0])
        
        # Rename columns to be more descriptive
        df.columns = [
            "State", "Total_Population", "Population_Less_than_9th_Grade", 
            "Population_9th_to_12th_No_Diploma", "Population_High_School_Graduate", 
            "Population_Some_College_No_Degree", "Population_Associate_Degree", 
            "Population_Bachelor_Degree", "Population_Master_Degree", 
            "Population_Professional_School_Degree", "Population_Doctorate_Degree", 
            "Population_Some_College_Less_1_year", "Population_Some_College_1_or_More_years_No_Degree",
            "Population_Associate_Degree", "Population_Bachelor_Degree", 
            "Population_Some_Degrees", "State_Code"
        ]

        # Add year column
        df['Year'] = '2021'
        
        # Save the DataFrame to CSV
        output_file = (
            r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find"
            r"\Downloaded_Data\Census_US_states_education.csv"
        )
        df.to_csv(output_file, index=False)
        print("Data successfully downloaded and saved.")
    else:
        print(f"Failed to fetch data. HTTP status code: {response.status_code}")

download_data()

In [13]:
# gdf = gpd.read_file(saved_fname)
# # gdf
# gdf.plot()

In [14]:
# wuhan_gdf.plot()
# railway_data.to_file(r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Wuhan_Railway_network.gpkg', driver='GPKG')

# # points
# railway_data

In [15]:
# place_name = 'Wuhan, Hubei, China'
# place_boundary = ox.geocode_to_gdf(place_name)

# # Retrieve the Bounding Box of Wuhan
# west, south, east, north = place_boundary.unary_union.bounds

# # Query String to Download Railway Data for the Bounding Box using Overpass API
# overpass_url = "https://overpass-api.de/api/interpreter"
# overpass_query = f"""
# [out:json];
# (way["railway"]( {south}, {west}, {north}, {east} );
# );
# out geom;
# """
# # Sending GET Request to Overpass API
# response = requests.get(overpass_url, 
#                         params={'data': overpass_query})

# # Parsing the JSON Response
# data = response.json()
# coords_list = []
# tags_list = []

# # Reading the Railway Network Data
# for element in data['elements']:
#     if 'geometry' in element:
#         coords = [(geom['lon'], geom['lat']) for geom in element['geometry']]
#         coords_list.append(LineString(coords))
#         tags_list.append(element['tags'])

# # Creating a GeoDataFrame
# railway_data = gpd.GeoDataFrame(tags_list, geometry=coords_list)

In [16]:

%autoreload 2
%load_ext autoreload
    
code = helper.execute_complete_program(code=code, try_cnt=10, task=task, model_name=model_name)
display(Code(code, language='python'))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


-------------- Running code (trial # 1/10) --------------


code in get_debug_prompt: import requests
import pandas as pd

def download_data():
    # API base URL
    base_url = "https://api.census.gov/data/2021/acs/acs1"
    
    # API key
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    
    # Specify the variables we are interested in
    # B15003_001E: Total Population (Education Attainment)
    # B15003_002E to B15003_017E: Population by specific education attainment (less than 9th grade to Doctorate degree)
    variables = [
        "NAME", "B15003_001E", "B15003_002E", "B15003_003E", "B15003_004E", 
        "B15003_005E", "B15003_006E", "B15003_007E", "B15003_008E", 
        "B15003_009E", "B15003_010E", "B15003_011E", "B15003_012E", 
        "B15003_013E", "B15003_014E", "B15003_015E", "B15003_016E", "B15003_017E"
    ]

    # Construct the query URL
    url = f"{base_url}?get=

import requests
import pandas as pd

def download_data():
    # API base URL
    base_url = "https://api.census.gov/data/2021/acs/acs1"
    
    # API key
    api_key = "b12026d61228a4b0d441ae7aa93f1ea222877503"
    
    # Specify the variables we are interested in
    variables = [
        "NAME", "B15003_001E", "B15003_002E", "B15003_003E", "B15003_004E", 
        "B15003_005E", "B15003_006E", "B15003_007E", "B15003_008E", 
        "B15003_009E", "B15003_010E", "B15003_011E", "B15003_012E", 
        "B15003_013E", "B15003_014E", "B15003_015E", "B15003_016E", "B15003_017E"
    ]

    # Construct the query URL
    url = f"{base_url}?get={','.join(variables)}&for=state:*&key={api_key}"

    # Request the data
    response = requests.get(url)
    if response.status_code == 200:
        # Convert to JSON
        data = response.json()
        
        # Create a DataFrame from the received data
        df = pd.DataFrame(data[1:], columns=data[0])
        
        # Rename columns to be more descriptive
        df.columns = [
            "State", "Total_Population", "Population_Less_than_9th_Grade", 
            "Population_9th_to_12th_No_Diploma", "Population_High_School_Graduate", 
            "Population_Some_College_No_Degree", "Population_Associate_Degree", 
            "Population_Bachelor_Degree", "Population_Master_Degree", 
            "Population_Professional_School_Degree", "Population_Doctorate_Degree", 
            "Population_Some_College_Less_1_year", "Population_Some_College_1_or_More_years_No_Degree", 
            "Population_Associate_Degree", "Population_Bachelor_Degree", 
            "Population_Master_Degree", "Population_Professional_School_Degree", 
            "Population_Doctorate_Degree", "State_Code"
        ]

        # Add year column
        df['Year'] = '2021'
        
        # Save the DataFrame to CSV
        output_file = (
            r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find"
            r"\Downloaded_Data\Census_US_states_education.csv"
        )
        df.to_csv(output_file, index=False)
        print("Data successfully downloaded and saved.")
    else:
        print(f"Failed to fetch data. HTTP status code: {response.status_code}")

download_data()

# Show the downloaded data

In [17]:
ext_name = saved_fname[-4:]
ext_name

'.csv'

In [18]:
if ext_name == '.csv':
    df = pd.read_csv(saved_fname)
    print(df)

if ext_name == 'gpkg':
    gdf = gpd.read_file(saved_fname)
    gdf.plot()
    print(gdf)

                   State  Total_Population  Population_Less_than_9th_Grade  \
0                Alabama           3451208                           51806   
1            Puerto Rico           2401409                           50439   
2                Arizona           4980297                           84990   
3               Arkansas           2037763                           31956   
4             California          26909869                          982757   
5               Colorado           4044182                           59127   
6            Connecticut           2534376                           47590   
7               Delaware            711104                            8844   
8   District of Columbia            478774                            9371   
9                Florida          15762122                          300597   
10               Georgia           7234271                          129927   
11                Hawaii           1021687                      

In [19]:
saved_fname

'E:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\LLM-Find\\Downloaded_Data\\Census_US_states_education.csv'

In [20]:
df.iloc[:, 1:-2]#.sum().sum()
df

,State,Total_Population,Population_Less_than_9th_Grade,Population_9th_to_12th_No_Diploma,Population_High_School_Graduate,Population_Some_College_No_Degree,Population_Associate_Degree,Population_Bachelor_Degree,Population_Master_Degree,Population_Professional_School_Degree,Population_Doctorate_Degree,Population_Some_College_Less_1_year,Population_Some_College_1_or_More_years_No_Degree,Population_Associate_Degree.1,Population_Bachelor_Degree.1,Population_Master_Degree.1,Population_Professional_School_Degree.1,Population_Doctorate_Degree.1,State_Code,Year
0,Alabama,3451208,51806,1043,450,746,3428,4084,2998,4885,13297,10684,33631,54057,75750,93942,65666,902759,1,2021
1,Puerto Rico,2401409,50439,1172,430,9236,14451,30222,32419,28877,56323,37067,48777,86255,30727,45203,17182,619884,72,2021
2,Arizona,4980297,84990,1581,928,2166,6293,10621,6464,10389,50426,12181,47599,65719,62639,72236,115131,967537,4,2021
3,Arkansas,2037763,31956,724,599,657,1778,3939,2500,2806,12686,10779,22236,29572,35353,40633,34672,572881,5,2021
4,California,26909869,982757,8764,15259,32424,61983,133468,83011,134582,599209,94116,235084,398827,275559,358556,771280,4909898,6,2021
5,Colorado,4044182,59127,320,668,823,2686,5840,2963,4977,27511,5980,24136,32769,31925,44762,61487,658973,8,2021
6,Connecticut,2534376,47590,1421,897,931,3104,3286,3187,8862,13296,6824,19040,25840,22533,25586,42794,568748,9,2021
7,Delaware,711104,8844,59,209,360,292,389,1287,881,3171,1045,6788,5300,8810,12490,11541,178620,10,2021
8,District of Columbia,478774,9371,116,215,167,733,245,593,598,2215,953,995,2540,5605,6067,4128,59776,11,2021
9,Florida,15762122,300597,5847,4167,6284,11102,29598,22459,39268,110247,46011,115539,159876,196926,205444,355178,3686864,12,2021


In [27]:
df['Total_Population'].sum()

230594873

In [21]:
# import geopandas as gpd
# import osmnx as ox
# import requests
# import os
# from shapely.geometry import LineString

# def download_data():
#     # Define the area of interest (Wuhan, Hubei Province, China)
#     place_name = "Wuhan, Hubei, China"
    
#     # First, get the boundary of Wuhan
#     boundaries = ox.geocode_to_gdf(place_name)
    
#     # Convert the boundary to a bounding box for the Overpass API query
#     bbox = boundaries.unary_union.bounds
#     north, south, east, west = bbox[3], bbox[1], bbox[2], bbox[0]
    
#     # Construct the Overpass API query for the railway network
#     overpass_url = "http://overpass-api.de/api/interpreter"
#     # Notice formatting, 'poly' requires coordinate pairs in clockwise order and close it by repeating the first point
#     overpass_query = f"""
#     [out:json];
#     (
#       way["railway"](poly:"{south} {west} {south} {east} {north} {east} {north} {west} {south} {west}");
#     );
#     out geom;
#     """

#     overpass_query = """   
#     [out:json];
#     (
#       way["railway"]( 29.971956, 113.076933, 31.362241, 115.696653);
#       relation["railway"]( 29.971956, 113.076933, 31.362241, 115.696653);
#       area["railway"]( 29.971956, 113.076933, 31.362241, 115.696653);
#     );
#     out geom;
#     """

#     print("overpass_query:", overpass_query)
    
#     # Send request to the Overpass API
#     response = requests.get(overpass_url, params={"data": overpass_query})
#     data = response.json()

#     print("data:", data.keys())

#     # return data
    
#     # Parse the response to get railway polylines
#     railways = []
#     geometries = []
#     for element in data['elements']:
#         if element['type'] == 'way' and 'geometry' in element:
#             coords = [(pt['lon'], pt['lat']) for pt in element['geometry']]
#             line = LineString(coords)
#             properties = element['tags']
#             railways.append(properties)
#             geometries.append(line)
    
#     # Convert railway data to GeoDataFrame, explicitly setting the geometry
#     gdf = gpd.GeoDataFrame(railways, crs="EPSG:4326", geometry=geometries)

#     # Define file path to save
#     file_path = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Wuhan_Railway_network.gpkg"

#     # Create directories if they do not exist
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)

#     # Save the data as a GeoPackage
#     gdf.to_file(file_path, layer='railways', driver="GPKG")

#     return data, gdf

# geo_json_dict, gdf = download_data()
# gdf.plot()

In [22]:
# # gdf
# elements = geo_json_dict['elements']

# len(elements)

In [23]:
# # convert 'way' to line
# from tqdm import tqdm

# def way_to_line(geo_json_dict):

#     return lines

# elements = geo_json_dict['elements']
# railways = []
# geometries = []
# way_cnt = 0
# rel_cnt = 0
# area_cnt = 0
# print(len(elements))
# for element in tqdm(elements):
#     try:
#             # print(type(element))
#         if element['type'] == 'way' and 'geometry' in element:
#             coords = [(pt['lon'], pt['lat']) for pt in element['geometry']]
#             # print(element['type'])
#             line = LineString(coords)
#             properties = element['tags']
#             railways.append(properties)
#             geometries.append(line) 
#             way_cnt += 1
            
#         if element['type'] == 'relation' and 'geometry' in element:
#                 rel_cnt += 1
            
#         if element['type'] == 'area' and 'geometry' in element:
#                 area_cnt += 1
            

#     except Exception as e:
#         print(e, element)

# way_cnt, rel_cnt, area_cnt

In [24]:
# import geopandas as gpd
# from shapely.geometry import LineString
# import requests

# def download_data():
#     # Query for Overpass API to get the railway network in Hubei Province, China
#     overpass_url = "http://overpass-api.de/api/interpreter"
#     overpass_query = """
#     [out:json];
#     area["name"="Hubei"]->.searchArea;
#     (
#       way["railway"](area.searchArea);
#     );
#     out body;
#     >;
#     out skel qt;
#     """
    
#     try:
#         response = requests.get(overpass_url, params={'data': overpass_query})
#         response.raise_for_status() # Check if the request was successful
#         data = response.json()
#     except requests.exceptions.RequestException as e:
#         print(f"Error downloading data: {e}")
#         return
    
#     # Parse the downloaded data
#     elements = data.get('elements', [])
#     if not elements:
#         print("No data found in the response.")
#         return
    
#     nodes = {element['id']: (element['lat'], element['lon']) for element in elements if element['type'] == 'node'}
#     ways = [element for element in elements if element['type'] == 'way']

#     # Creating GeoDataFrame
#     features = []
#     for way in ways:
#         try:
#             coords = [nodes[node] for node in way['nodes']]
#             if 'tags' in way:
#                 tags = way['tags']
#                 feature = {
#                     'type': 'Feature',
#                     'geometry': LineString(coords),
#                     'properties': tags
#                 }
#                 features.append(feature)
#         except KeyError as e:
#             # If a node ID in 'nodes' does not exist in 'nodes' dictionary, skip the way
#             print(f"Skipping way with missing node: {e}")
#             continue
    
#     geojson_data = {'type': 'FeatureCollection', 'features': features}
    
#     gdf = gpd.GeoDataFrame.from_features(geojson_data['features'], crs="EPSG:4326")
    
#     # Save as GeoPackage
#     output_path = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Hubei_Railway_network.gpkg"
#     gdf.to_file(output_path, layer='railways', driver="GPKG")

# download_data()

# gdf = gpd.read_file(saved_fname)
# gdf.plot()

In [25]:
# import osmnx as ox
# import geopandas as gpd

# def download_boundary(city_name):
#     """
#     Download the administrative boundary of a specified city using OSMnx.

#     Parameters:
#     - city_name (str): Name of the city to download the boundary for.

#     Returns:
#     - GeoDataFrame containing the city boundary.
#     """
#     # Specify the city and the boundary type (administrative)
#     place_query = {'city': city_name, 'country': 'China'}
#     boundary_gdf = ox.geocode_to_gdf(place_query, which_result=2)
    
#     # Optionally, save the boundary to a file
#     boundary_gdf.to_file(f"{city_name}_boundary.shp")

#     return boundary_gdf

# # Example usage: Download and plot the boundary of Wuhan
# wuhan_boundary = download_boundary("Wuhan")
# print(wuhan_boundary)

# # Plot the boundary
# wuhan_boundary.plot()